In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from embeddings.dataloader import TheDataSet
%matplotlib inline
import matplotlib.pyplot as plt
# import tensorboard
# from tensorboard import SummaryWriter



In [3]:
encoder_training_epochs = 150
# dataset_file = 'data_ae_train/fulldata.npy'
# dataset_file = 'data/fulldata.npy'
dataset_file = 'data/autoencoder_fulldata.npy'
# dataset_file = 'data/autoencoder_fulldata_mini.npy'

# dataset_file = 'data/labdata.npy'

In [4]:
mxxx = np.load(dataset_file)

In [5]:
mxxx.shape, np.max(mxxx), np.min(mxxx)

((54871, 1338), 1.0, 0.0)

In [6]:
del mxxx

In [6]:
# default `log_dir` is "runs" - we'll be more specific here
# writer = SummaryWriter(f"runs/autoencoder_experiment_{encoder_training_epochs}")


## AutoEncoder


In [4]:
%%writefile embeddings/autoencoder.py

import torch
import torch.nn as nn


class Autoencoder(nn.Module):
    def __init__(self, num_features):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_features=num_features, out_features=num_features // 2 , bias=True),
            nn.Dropout(p = 0.1),
            nn.Tanh(),
            nn.Linear(in_features=num_features // 2, out_features=128, bias=True),
            nn.Tanh(),
            nn.Linear(in_features=128, out_features=64, bias=True),
            nn.Tanh()
        )
        self.decoder = nn.Sequential(
            nn.Linear(in_features=64, out_features=128, bias=True),
            nn.Tanh(),
            nn.Linear(in_features=128, out_features=num_features // 2, bias=True),
            nn.Tanh(),
            nn.Linear(in_features=num_features // 2, out_features=num_features, bias=True ),
            #nn.Sigmoid()
            nn.Tanh()
#             nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


Overwriting embeddings/autoencoder.py


In [3]:
%%writefile embeddings/autoencoder.py

import torch
import torch.nn as nn


class Autoencoder(nn.Module):
    def __init__(self, num_features):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(in_features=num_features, out_features=128 , bias=True),
            nn.Tanh(),
            nn.Linear(in_features=128, out_features=64, bias=True),
            nn.Tanh(),
#             nn.Linear(in_features=64, out_features=32, bias=True),
#             nn.Tanh()
        )
        self.decoder = nn.Sequential(
#             nn.Linear(in_features=32, out_features=64, bias=True),
#             nn.Tanh(),
            nn.Linear(in_features=64, out_features=128, bias=True),
            nn.Tanh(),
            nn.Linear(in_features=128, out_features=num_features, bias=True ),
            #nn.Sigmoid()
            nn.Tanh()
#             nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


Overwriting embeddings/autoencoder.py


In [5]:
from embeddings.autoencoder import Autoencoder

## Training

In [6]:
def train(model, dataset, num_epochs=5, batch_size=64, learning_rate=1e-3):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = model.to(device)
    torch.manual_seed(42)
    criterion = nn.MSELoss() # mean square error loss
#     criterion = nn.L1Loss()
#     criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(),
                                 lr=learning_rate, 
                                 weight_decay=1e-5) # <--
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    outputs = []
    losses = []
    for epoch in range(num_epochs):
        for X, y in train_loader:
            X = X.to(device)
            recon = model(X.float())
            loss = criterion(recon, X.float())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        print('Epoch:{}, Loss:{:.4f}'.format(epoch+1, float(loss)))
        losses.append(float(loss))
#         writer.add_scalar(f'training loss {num_epochs}',
#                             loss,
#                             epoch * len(train_loader))
        # stop training if loss is low
        if loss < 0.0050:
            break
    return outputs, losses

In [7]:
dataset = TheDataSet(datafile=dataset_file, pad_to_360=False)
print(f"dataset length = {len(dataset)} num features = {dataset.num_features()}")
model = Autoencoder(num_features=dataset.num_features())
print(model)
max_epochs = encoder_training_epochs

dataset length = 54871 num features = 1337
Autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=1337, out_features=668, bias=True)
    (1): Dropout(p=0.1, inplace=False)
    (2): Tanh()
    (3): Linear(in_features=668, out_features=128, bias=True)
    (4): Tanh()
    (5): Linear(in_features=128, out_features=64, bias=True)
    (6): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=64, out_features=128, bias=True)
    (1): Tanh()
    (2): Linear(in_features=128, out_features=668, bias=True)
    (3): Tanh()
    (4): Linear(in_features=668, out_features=1337, bias=True)
    (5): Tanh()
  )
)


In [ ]:
%%time
outputs, losses = train(model, dataset=dataset, num_epochs=max_epochs, batch_size = 512, learning_rate=1e-3)

Epoch:1, Loss:0.0458
Epoch:2, Loss:0.0325
Epoch:3, Loss:0.0282
Epoch:4, Loss:0.0262
Epoch:5, Loss:0.0245
Epoch:6, Loss:0.0225
Epoch:7, Loss:0.0211
Epoch:8, Loss:0.0191
Epoch:9, Loss:0.0194
Epoch:10, Loss:0.0192
Epoch:11, Loss:0.0190
Epoch:12, Loss:0.0184
Epoch:13, Loss:0.0164
Epoch:14, Loss:0.0169
Epoch:15, Loss:0.0160
Epoch:16, Loss:0.0159
Epoch:17, Loss:0.0162
Epoch:18, Loss:0.0159
Epoch:19, Loss:0.0158
Epoch:20, Loss:0.0145
Epoch:21, Loss:0.0151
Epoch:22, Loss:0.0129
Epoch:23, Loss:0.0149
Epoch:24, Loss:0.0165
Epoch:25, Loss:0.0149
Epoch:26, Loss:0.0153
Epoch:27, Loss:0.0147
Epoch:28, Loss:0.0143
Epoch:29, Loss:0.0152
Epoch:30, Loss:0.0144
Epoch:31, Loss:0.0133
Epoch:32, Loss:0.0141
Epoch:33, Loss:0.0137
Epoch:34, Loss:0.0158
Epoch:35, Loss:0.0147
Epoch:36, Loss:0.0141
Epoch:37, Loss:0.0155
Epoch:38, Loss:0.0135
Epoch:39, Loss:0.0143
Epoch:40, Loss:0.0144
Epoch:41, Loss:0.0132
Epoch:42, Loss:0.0144
Epoch:43, Loss:0.0142
Epoch:44, Loss:0.0137
Epoch:45, Loss:0.0140
Epoch:46, Loss:0.01

In [ ]:
fig = plt.figure()
ax = plt.axes()

# x = np.linspace(0, 10, 1000)
ax.plot(losses);

In [73]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset = TheDataSet(datafile='data/fulldata.npy', pad_to_360=False)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False)
rows=[]
for X, y in data_loader:
    X = X.to(device)
    y = y.to(device)
    row = model.encoder(X.float())
    row = torch.cat([row.reshape(1,-1),y.reshape(1,-1).float()], dim=1)
    rows.append(row)

In [74]:
len(rows)

4085

In [75]:
encoded_data = torch.cat(rows, dim=0)

In [76]:
np_labeled_data = encoded_data.detach().to('cpu').numpy()

In [77]:
np.save('data/np_autoencoded_labeled_data.npy', np_labeled_data)

In [72]:
row.reshape(1,-1).shape, torch.cat([row.reshape(1,-1),y.reshape(1,-1).float()], dim=1)

(torch.Size([1, 65]),
 tensor([[ 0.4013, -0.6855, -0.7156,  0.9339, -0.9558,  0.5300, -0.0857,  0.7046,
           0.2912,  0.7087, -0.8098, -0.8930, -0.5535,  0.5503, -0.9506,  0.4161,
           0.0843, -0.4683,  0.4343, -0.5010, -0.4567, -0.0906,  0.5301,  0.7316,
           0.9533, -0.2443,  0.7769,  0.0584, -0.7406, -0.7253, -0.7416, -0.0751,
          -0.3182, -0.3416,  0.7200, -0.5329, -0.6897, -0.0160,  0.1136, -0.4463,
           0.3447, -0.5107, -0.6689,  0.3179, -0.1100, -0.1150,  0.0790,  0.7705,
          -0.3329,  0.6714, -0.7256,  0.4391,  0.1403,  0.4815, -0.6231, -0.5074,
           0.0524,  0.6013, -0.5924,  0.4243,  0.1845, -0.0379,  0.0665, -0.2875,
           0.0000,  0.0000]], device='cuda:0', grad_fn=<CatBackward>))

In [ ]:
import numpy as np
np.save()

In [14]:
min(losses)

0.012076033279299736

In [15]:
# persist the encoder


with open('data/autoencoder.pic', 'bw') as f:
    torch.save(model, f, pickle_protocol=4)




In [10]:
with open('data/autoencoder.pic', 'rb') as f:
    autoencoder = torch.load(f)